# IRWA Assignment - 03

Group Members :
* IT18215502 - K.Priyanka 
* IT18217896 - M.Mayuriya



* Topic : 
    - Sentiment Analysis of IMDB Movie Reviews

* What we are planning to focus on:
    - In this, we are planning to predict the number of positive and negative reviews  based on sentiments of movies(Bollywood Industry) that have been released through online platforms during quarantine (March - September) using different classification models.

#### Importing the needed libraries

In [17]:
#Load the libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelBinarizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from wordcloud import WordCloud,STOPWORDS
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize,sent_tokenize
from bs4 import BeautifulSoup
import spacy
import re,string,unicodedata
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.stem import LancasterStemmer,WordNetLemmatizer
from sklearn.linear_model import LogisticRegression,SGDClassifier
from sklearn.naive_bayes import MultinomialNB

In [18]:
from sklearn.svm import SVC
from textblob import TextBlob
from textblob import Word
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score

#### Reading Scraped Dataset

In [224]:
imdb_data=pd.read_csv('userReviews.csv')
print(imdb_data.shape)

(86, 4)


In [225]:
imdb_data.head(10)

,movie,user_review_permalink,user_review,sentiment
0,Dil Bechara,https://www.imdb.com/review/rw5963527/,SSR has done some amazing work in some amazing...,negative
1,Dil Bechara,https://www.imdb.com/review/rw5930232/,One of the finest acting I've ever seen.I sugg...,positive
2,Sadak 2,https://www.imdb.com/review/rw6035756/,"The story does not make sense, the action scen...",negative
3,Sadak 2,https://www.imdb.com/review/rw6035756/,"The story does not make sense, the action scen...",positive
4,London Confidental,https://www.imdb.com/review/rw6108883/,It's not bad as they are showcaseing above. Th...,negative
5,London Confidental,https://www.imdb.com/review/rw6116067/,Amid a series of brutal killings of Indian age...,positive
6,Halahal,https://www.imdb.com/review/rw6113125/,Halahal begins and ends with scenes set in the...,negative
7,Halahal,https://www.imdb.com/review/rw6114491/,With so many movies steaming from different so...,positive
8,Raat Akeli Hai,https://www.imdb.com/review/rw5955736/,"All performances are good, casting is good, bu...",negative
9,Raat Akeli Hai,https://www.imdb.com/review/rw5955295/,Nawazuddin and Radhika!! Two gems of Bollywood...,positive


In [226]:
imdb_data['sentiment'].value_counts()

positive    43
negative    43
Name: sentiment, dtype: int64

#### Downloading the stopwords and wordnet NLTK files

In [227]:
#from nltk.corpus import stopwords
nltk.download("stopwords")
nltk.download("wordnet")

#applying stopwords to words in english language
stopword_list=stopwords.words('english')

[nltk_data] Downloading package stopwords to C:\Users\Priyanka
[nltk_data]     Kugapriya\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\Priyanka
[nltk_data]     Kugapriya\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


#### Spliting the dataset into train dataset and test dataset

In [228]:
#train dataset
train_reviews=imdb_data.user_review[:65]
train_sentiments=imdb_data.sentiment[:65]

#test dataset
test_reviews=imdb_data.user_review[65:]
test_sentiments=imdb_data.sentiment[65:]

#print statement
print(train_reviews.shape,train_sentiments.shape)
print(test_reviews.shape,test_sentiments.shape)

(65,) (65,)
(21,) (21,)


#### Normalization of the text

In [229]:
#Tokenization of text

tokenizer=ToktokTokenizer()
#imdb_data.apply(tokenizer.tokenize)


#Setting English stopwords
stopword_list=nltk.corpus.stopwords.words('english')

#### Clean the datasets such as removing any special characters, html tags, etc

In [230]:
#Removing the html strips
def strip_html(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

#Removing the square brackets
def remove_between_square_brackets(text):
    return re.sub('\[[^]]*\]', '', text)


In [231]:
#Removing the noisy text
def denoise_text(text):
    text = strip_html(text)
    text = remove_between_square_brackets(text)
    return text

#Applying the function on review column
imdb_data['user_review']=imdb_data['user_review'].apply(denoise_text)

In [232]:
#Define function for removing special characters
def remove_special_characters(text, remove_digits=True):
    pattern=r'[^a-zA-z0-9\s]'
    text=re.sub(pattern,'',text)
    return text

#Applying the function on review column
imdb_data['user_review']=imdb_data['user_review'].apply(remove_special_characters)

#### Stemming

In [233]:
#Stemming the text
def simple_stemmer(text):
    ps=nltk.porter.PorterStemmer()
    text= ' '.join([ps.stem(word) for word in text.split()])
    return text

#Applying the function on review column
imdb_data['user_review']=imdb_data['user_review'].apply(simple_stemmer)

#### Lemmatization

In [234]:
#lemmatization of the texts
    #from nltk.stem import WordNetLemmatizer
    #lemmatizer = nltk.stem.WordNetLemmatizer()

#Applying the function on review column
    #imdb_data['user_review']=imdb_data['user_review'].apply(lambda x: [lemmatizer.lemmatize(word) for word in x])

#### Removing Stopwords

In [235]:
#set stopwords to english
stop=set(stopwords.words('english'))

#removing the stopwords
def remove_stopwords(text, is_lower_case=False):
    tokens = tokenizer.tokenize(text)
    tokens = [token.strip() for token in tokens]
    if is_lower_case:
        filtered_tokens = [token for token in tokens if token not in stopword_list]
    else:
        filtered_tokens = [token for token in tokens if token.lower() not in stopword_list]
    filtered_text = ' '.join(filtered_tokens)    
    return filtered_text

In [236]:
#Applying the function on review column
imdb_data['user_review']=imdb_data['user_review'].apply(remove_stopwords)

In [237]:
#lemmatization of the texts
    #from nltk.stem import WordNetLemmatizer
    #lemmatizer = nltk.stem.WordNetLemmatizer()

#Applying the function on review column
    #imdb_data['user_review']=imdb_data['user_review'].apply(lambda x: [lemmatizer.lemmatize(word) for word in x])

#### Normalized Train Reviews

In [238]:
#normalized train reviews
norm_train_reviews=imdb_data.user_review[:65]
norm_train_reviews[0]


'ssr ha done amaz work amaz movi thi one yeah charm fun watch like alway hi best movi gave thi movi 1010 base guilt guilt never watch hi movi actual worthi 1010 better movi thi dont go gener rate thi watch sonchiriya kai po che chhichhor'

#### Normalized Test Reviews

In [191]:
#Normalized test reviews
norm_test_reviews=imdb_data.user_review[65:]
norm_test_reviews[70]


'wast 2 hour life never get back dont watch thi movi even prison'

#### Bags of Words Model

In [192]:
#Count vectorizer for bag of words
cv=CountVectorizer(min_df=0,max_df=1,binary=False,ngram_range=(1,3))

#transformed train reviews
cv_train_reviews=cv.fit_transform(norm_train_reviews)

#transformed test reviews
cv_test_reviews=cv.transform(norm_test_reviews)

#print statement
print('BOW_cv_train:',cv_train_reviews.shape)
print('BOW_cv_test:',cv_test_reviews.shape)

BOW_cv_train: (65, 6223)
BOW_cv_test: (21, 6223)


#### Term Frequency-Inverse Document Frequency model (TFIDF)

In [193]:
#Tfidf vectorizer
tv=TfidfVectorizer(min_df=0,max_df=1,use_idf=True,ngram_range=(1,3))

#transformed train reviews
tv_train_reviews=tv.fit_transform(norm_train_reviews)

#transformed test reviews
tv_test_reviews=tv.transform(norm_test_reviews)

#print statement
print('Tfidf_train:',tv_train_reviews.shape)
print('Tfidf_test:',tv_test_reviews.shape)

Tfidf_train: (65, 6223)
Tfidf_test: (21, 6223)


#### Labeling the sentiment text (Target variable)

In [194]:
#labeling the sentiment data
lb=LabelBinarizer()

#transformed sentiment data
sentiment_data=lb.fit_transform(imdb_data['sentiment'])

#print statement
print(sentiment_data.shape)

(86, 1)


#### Spliting the sentiment dataset into train dataset and test dataset

In [195]:
#Spliting the sentiment data
train_sentiments=sentiment_data[:65]
test_sentiments=sentiment_data[65:]

#print statement
print(train_sentiments)
print(test_sentiments)

[[0]
 [1]
 [0]
 [1]
 [0]
 [1]
 [0]
 [1]
 [0]
 [1]
 [0]
 [1]
 [0]
 [1]
 [0]
 [1]
 [0]
 [1]
 [0]
 [1]
 [0]
 [1]
 [0]
 [1]
 [0]
 [1]
 [0]
 [1]
 [0]
 [1]
 [0]
 [1]
 [0]
 [1]
 [0]
 [1]
 [0]
 [1]
 [0]
 [1]
 [0]
 [1]
 [0]
 [1]
 [0]
 [1]
 [0]
 [1]
 [0]
 [1]
 [0]
 [1]
 [0]
 [1]
 [0]
 [1]
 [0]
 [1]
 [0]
 [1]
 [0]
 [1]
 [0]
 [1]
 [0]]
[[1]
 [0]
 [1]
 [0]
 [1]
 [0]
 [1]
 [0]
 [1]
 [0]
 [1]
 [0]
 [1]
 [0]
 [1]
 [0]
 [1]
 [0]
 [1]
 [0]
 [1]]


### Building the Multinominal Naive Bayes model using the Train data

In [196]:
#training the model
mnb=MultinomialNB()

#fitting the svm for bag of words
mnb_bow=mnb.fit(cv_train_reviews,train_sentiments)
#print statement
print(mnb_bow)

#fitting the svm for tfidf features
mnb_tfidf=mnb.fit(tv_train_reviews,train_sentiments)
#print statement
print(mnb_tfidf)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)
MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)


C:\Users\Priyanka Kugapriya\anaconda3\lib\site-packages\sklearn\utils\validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Priyanka Kugapriya\anaconda3\lib\site-packages\sklearn\utils\validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


### Testing the model using Test Data

In [197]:
#Predicting the model for bag of words
mnb_bow_predict=mnb.predict(cv_test_reviews)
#print statement
print(mnb_bow_predict)

#Predicting the model for tfidf features
mnb_tfidf_predict=mnb.predict(tv_test_reviews)
#print statement
print(mnb_tfidf_predict)

[0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 1 1 1 1 0 0]
[0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 1 1 1 1 0 0]


In [198]:
#Accuracy score for bag of words
mnb_bow_score=accuracy_score(test_sentiments,mnb_bow_predict)
#print statement
print("mnb_bow_score :",mnb_bow_score)

#Accuracy score for tfidf features
mnb_tfidf_score=accuracy_score(test_sentiments,mnb_tfidf_predict)
#print statement
print("mnb_tfidf_score :",mnb_tfidf_score)

mnb_bow_score : 0.5714285714285714
mnb_tfidf_score : 0.5714285714285714


### Calculating the confusion matrix and identifying the model performance.

In [199]:
#confusion matrix for bag of words
cm_bow=confusion_matrix(test_sentiments,mnb_bow_predict,labels=[1,0])
#print statement
print(cm_bow)

#confusion matrix for tfidf features
cm_tfidf=confusion_matrix(test_sentiments,mnb_tfidf_predict,labels=[1,0])
#print statement
print(cm_tfidf)

[[4 7]
 [2 8]]
[[4 7]
 [2 8]]


### Building the K-NN model using the Train data and a random K value

In [200]:
from sklearn.neighbors import KNeighborsClassifier

#training the KNN
KNN = KNeighborsClassifier(n_neighbors = 5)

#fitting the knn for bag of words
knn_bow = KNN.fit(cv_train_reviews,train_sentiments)
#print statement
print(knn_bow)

#fitting the knn for tfidf features
y_pred =KNN.predict(tv_test_reviews)
#print statement
print(y_pred)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]


C:\Users\Priyanka Kugapriya\anaconda3\lib\site-packages\ipykernel_launcher.py:7: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  import sys


### Testing the model using Test Data

In [201]:
cm_test =confusion_matrix(test_sentiments,y_pred)*100

In [202]:
#Accuracy score
accuracy_score(test_sentiments,y_pred)

0.5238095238095238

## Calculating the confusion matrix and identifying the model performance.

In [203]:
confusion_matrix(cm_test,acc,model)

NameError: name 'acc' is not defined

### Build SVM model using Train Data

In [204]:
#training the svm
svm=SGDClassifier(loss='hinge',max_iter=500,random_state=42)

#fitting the svm for bag of words
svm_bow=svm.fit(cv_train_reviews,train_sentiments)
#print statement
print(svm_bow)

#fitting the svm for tfidf features
svm_tfidf=svm.fit(tv_train_reviews,train_sentiments)
#print statement
print(svm_tfidf)

SGDClassifier(alpha=0.0001, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
              l1_ratio=0.15, learning_rate='optimal', loss='hinge',
              max_iter=500, n_iter_no_change=5, n_jobs=None, penalty='l2',
              power_t=0.5, random_state=42, shuffle=True, tol=0.001,
              validation_fraction=0.1, verbose=0, warm_start=False)
SGDClassifier(alpha=0.0001, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
              l1_ratio=0.15, learning_rate='optimal', loss='hinge',
              max_iter=500, n_iter_no_change=5, n_jobs=None, penalty='l2',
              power_t=0.5, random_state=42, shuffle=True, tol=0.001,
              validation_fraction=0.1, verbose=0, warm_start=False)


C:\Users\Priyanka Kugapriya\anaconda3\lib\site-packages\sklearn\utils\validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


### Test the model using the test data

In [205]:
#Predicting the model for bag of words
svm_bow_predict=svm.predict(cv_test_reviews)
#print statement
print(svm_bow_predict)

#Predicting the model for tfidf features
svm_tfidf_predict=svm.predict(tv_test_reviews)
#print statement
print(svm_tfidf_predict)

[0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 1 1 0 0]
[0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 1 1 0 0]


In [206]:
#Accuracy score for bag of words
svm_bow_score=accuracy_score(test_sentiments,svm_bow_predict)
#print statement
print("svm_bow_score :",svm_bow_score)

#Accuracy score for tfidf features
svm_tfidf_score=accuracy_score(test_sentiments,svm_tfidf_predict)
#print statement
print("svm_tfidf_score :",svm_tfidf_score)

svm_bow_score : 0.5714285714285714
svm_tfidf_score : 0.5714285714285714


### Calculate the confusion matrix and identify the model performance

In [207]:
#confusion matrix for bag of words
cm_bow=confusion_matrix(test_sentiments,svm_bow_predict,labels=[1,0])
#print statement
print(cm_bow)

#confusion matrix for tfidf features
cm_tfidf=confusion_matrix(test_sentiments,svm_tfidf_predict,labels=[1,0])
#print statement
print(cm_tfidf)

[[3 8]
 [1 9]]
[[3 8]
 [1 9]]


#### Compare the model performance and select a best model and briefly discuss why this is the best model

* Looking at the above models it can be clearly seen that Support vector machine and the Multinominal Naive bayes model perform well when compared to K-nearest neighbor.
* But when considering the confusion matrix of Support vector machine and the Multinominal Naive bayes model the SVMs' false negative is lower than Multinominal Naive bayes model. 
* So as a conclusion I select the SVMs' model as the best model.


#### How can you improve the performance of your models?
* Having a dataset with a large number of data can increase the accuracy and improve the performance of the model
* Removing outlier values in the training data , because it causes to reduce the accuracy of a model. It also leads to inaccurate predictions
* We can imporve the accuracy by preprocessing data and by using models like Textblob.